In [1]:
import pandas as pd
import numpy as np

data = pd.DataFrame.from_csv('data.csv')
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data['text'].values, data['label'].values,
                                                    test_size=0.2, random_state=42)

In [2]:
import re

def get_words(line):
    return [x for x in re.split('[_|:\n ,.?!@$%^&*();"\'\/\\0-9\{2}\\\<>+=_\-]', line.lower().strip()) if x != '']

def append_words_from_lines(words, lines):
    for line in lines:
        for word in get_words(line):
            if word not in words and word:
                words.append(word)

vocab = []

append_words_from_lines(vocab, X_train)
append_words_from_lines(vocab, X_test)

In [3]:
len(vocab)

12823

In [4]:
print(vocab[:300])

['it', 'is', 'so', 'much', 'easier', 'for', 'me', 'to', 'write', 'how', 'i', 'm', 'feeling', 'than', 'actually', 'speak', 'get', 'nervous', 'and', 'stumble', 'on', 'my', 'words', 'if', 's', 'heart', 'watching', 'grandparents', 'parents', 'pay', 'all', 'these', 'bills', 'really', 'excites', 'the', 'future', 'godisalivenrock', 'ao', 'langshughes', 'greatest', 'unsolved', 'mystery', 'god', 'sysdig', 'cloud', 'fascinating', 'world', 'of', 'linux', 'system', 'calls', 'reminds', 'solaris', 'dtrace', 'days', 'in', 'early', 'http', 't', 'co', 'e', 'yyx', 'tgq', 'just', 'found', 'out', 'there', 'are', 'etch', 'a', 'sketch', 'apps', 'oldschool', 'notoldschool', 'coltonwilliy', 'im', 'sorry', 'chode', 'phone', 'broke', 'new', 'year', 'about', 'start', 'many', 'people', 'came', 'went', 'but', 'always', 'wat', 'gone', 'better', 'ummm', 'can', 'please', 'have', 'kipmooremusic', 'christmas', 'dang', 'he', 'looks', 'great', 'tonight', 'why', 'did', 'not', 'go', 'this', 'thing', 'philoliverh', 'nat', '

In [5]:
import word2vecReader
import numpy as np

w2v = word2vecReader.Word2Vec.load_word2vec_format('word2vec_twitter_model/word2vec_twitter_model.bin', binary=True)

In [6]:
not_contained = []
for word in vocab:
    if not w2v.__contains__(word):

        not_contained.append(word)

print(len(not_contained))
print(len(not_contained) / len(vocab))
print(not_contained[:300])

3530
0.27528659440068626
['godisalivenrock', 'langshughes', 'sysdig', 'yyx', 'tgq', 'notoldschool', 'coltonwilliy', 'kipmooremusic', 'philoliverh', 'bardgin', 'favthings', 'rsjucgrhjv', 'neilby', 'fionan', 'watchthelordman', 'hzdjzceaod', 'perksofbeingasenior', 'rikkiblack', 'sirtomhunter', 'mariemacklin', 'royalawesome', 'pdpnigeria', 'ccriadoperez', 'energypeople', 'chxta', 'thechrisstuckey', 'pai̇n', 'gai̇n', 'sudphibdez', 'coldpants', 'offtomysteriousplaces', 'aola', 'justacoolcat', 'antoineraps', 'edifyin', 'blakenath', 'icouldsleepforeverrightnow', 'emwatson', 'elleuk', 'lorraineelle', 'morgankiro', 'youoweme', 'txdiptwa', 'ovx', 'ttwon', 'peshawarattack', 'ptikeptpakfirst', 'araancheta', 'dewaardsara', 'llgp', 'uerns', 'ucom', 'jcrsursvld', 'bcvzigfnks', 'dcrising', 'wizrockets', 'hhao', 'coercivecontrol', 'wjfiq', 'torturereport', 'talkhoops', 'internetcelebrity', 'jimmyk', 'nickbaumann', 'uklabour', 'pmlive', 'interesinglife', 'whatamidoingwithmylife', 'longesthashtagnotneeded

In [7]:
print(w2v.__contains__('braw'))

True


In [8]:
x = [0 for _ in range(101)]

for line in X_train:
    l = len(get_words(line))
    if l < 100:
        x[l] += 1
        
for i, c in enumerate(x):
    print(i, c)

0 0
1 3
2 6
3 18
4 35
5 55
6 91
7 124
8 127
9 137
10 147
11 149
12 165
13 147
14 177
15 156
16 139
17 156
18 154
19 148
20 190
21 136
22 156
23 113
24 97
25 61
26 66
27 51
28 17
29 10
30 8
31 7
32 5
33 1
34 1
35 2
36 1
37 1
38 1
39 1
40 1
41 1
42 1
43 1
44 1
45 0
46 0
47 0
48 0
49 0
50 0
51 0
52 1
53 0
54 0
55 1
56 0
57 0
58 0
59 0
60 0
61 0
62 0
63 0
64 0
65 0
66 0
67 0
68 0
69 0
70 0
71 0
72 0
73 0
74 0
75 0
76 0
77 0
78 0
79 0
80 0
81 0
82 0
83 0
84 0
85 0
86 0
87 0
88 0
89 0
90 0
91 0
92 0
93 0
94 0
95 0
96 0
97 0
98 0
99 0
100 0


In [9]:
word_to_vec = {}
for word in vocab:
    if w2v.__contains__(word):
        word_to_vec[word] = w2v[word]
    else:
        word_to_vec[word] = np.zeros(400)
    

In [10]:
del w2v

In [11]:
import torch

seq_len = 30


X_train_vec_seq = []
for i, line in enumerate(X_train[::]):
    X_train_vec_seq.append([])
    words = get_words(line)
    
    for word in words[:seq_len]:
        X_train_vec_seq[-1].append(word_to_vec[word]) 
    while len(X_train_vec_seq[-1]) < seq_len:
        X_train_vec_seq[-1].append(np.zeros(400))
    
    
#     for j in range(seq_len):
#         if j < len(words):
#             X_train_vec_seq[-1].append(word_to_vec[words[j]])
#         else:
#             X_train_vec_seq[-1].append(np.zeros(400))
X_train_vec_seq = torch.from_numpy(np.array(X_train_vec_seq)).type(torch.FloatTensor)
            
X_test_vec_seq = []
for i, line in enumerate(X_test[::]):
    X_test_vec_seq.append([])
    words = get_words(line)
    
    for word in words[:seq_len]:
        X_test_vec_seq[-1].append(word_to_vec[word])
    while len(X_test_vec_seq[-1]) < seq_len:
        X_test_vec_seq[-1].append(np.zeros(400))
    
    
#     for j in range(seq_len):
#         if j < len(words):
#             X_test_vec_seq[-1].append(word_to_vec[words[j]])
#         else:
#             X_test_vec_seq[-1].append(np.zeros(400))
X_test_vec_seq = torch.from_numpy(np.array(X_test_vec_seq)).type(torch.FloatTensor)

y_train_torch = torch.from_numpy(np.array(y_train))   
y_test_torch = torch.from_numpy(np.array(y_test))

In [12]:
#print(X_train_vec_seq[1])

In [13]:
import torch 
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

In [14]:
# Hyper Parameters
sequence_length = seq_len
input_size = 400
hidden_size = 64
num_layers = 2
num_classes = 2
batch_size = len(X_train_vec_seq)
num_epochs = 500
learning_rate = 0.01

In [15]:
# RNN Model (Many-to-One)
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True).cuda()
        self.fc1 = nn.Linear(hidden_size, hidden_size).cuda()
        self.fc2 = nn.Linear(hidden_size, num_classes).cuda()
        self.log_softmax = nn.Softmax().cuda()
    
    def forward(self, x):
        # Set initial states 
        h0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).cuda()
        c0 = Variable(torch.zeros(self.num_layers, x.size(0), self.hidden_size)).cuda()
        
        # Forward propagate RNN
        out, _ = self.lstm(x, (h0, c0))  
        
        # Decode hidden state of last time step
        out = self.fc1(out[:, -1, :])
        out = self.fc2(out)
        out = self.log_softmax(out)
        return out

rnn = RNN(input_size, hidden_size, num_layers, num_classes)
rnn.cuda()

RNN (
  (lstm): LSTM(400, 64, num_layers=2, batch_first=True)
  (fc1): Linear (64 -> 64)
  (fc2): Linear (64 -> 2)
  (log_softmax): Softmax ()
)

In [16]:
# Loss and Optimizer
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.Adagrad(rnn.parameters(), lr = learning_rate)

In [19]:
optimizer.lr = 0.0001

In [20]:
# Train the Model
for epoch in range(num_epochs * 10):
        X_train_vec_seq = X_train_vec_seq.cuda()
        tweet = Variable(X_train_vec_seq)
        y_train_torch = y_train_torch.cuda()
        label = Variable(y_train_torch.view(-1))

        
    
        optimizer.zero_grad()
        outputs = rnn(tweet)

        loss = criterion(outputs, label)
        loss.backward()
        optimizer.step()
        
        if (epoch+1) % 10 == 0:
            print ('Epoch [%d/%d], Loss: %.4f' 
                   %(epoch+1, num_epochs, loss.data[0]))
            
        if loss.data[0] < 0.001:
            print('done')
            break

Epoch [10/500], Loss: 0.4372
Epoch [20/500], Loss: 0.4372
Epoch [30/500], Loss: 0.4372
Epoch [40/500], Loss: 0.4372
Epoch [50/500], Loss: 0.4372
Epoch [60/500], Loss: 0.4372
Epoch [70/500], Loss: 0.4372
Epoch [80/500], Loss: 0.4372
Epoch [90/500], Loss: 0.4372
Epoch [100/500], Loss: 0.4372
Epoch [110/500], Loss: 0.4372
Epoch [120/500], Loss: 0.4372
Epoch [130/500], Loss: 0.4372
Epoch [140/500], Loss: 0.4372
Epoch [150/500], Loss: 0.4372
Epoch [160/500], Loss: 0.4372
Epoch [170/500], Loss: 0.4372
Epoch [180/500], Loss: 0.4372
Epoch [190/500], Loss: 0.4372
Epoch [200/500], Loss: 0.4372
Epoch [210/500], Loss: 0.4372
Epoch [220/500], Loss: 0.4372
Epoch [230/500], Loss: 0.4372
Epoch [240/500], Loss: 0.4372
Epoch [250/500], Loss: 0.4372
Epoch [260/500], Loss: 0.4372
Epoch [270/500], Loss: 0.4372


KeyboardInterrupt: 

In [21]:
def score_model(model, X, y, model_name):
    from sklearn.metrics import roc_auc_score

    correct = 0
    total = 0
    positive = 0
    correct_positive = 0
    should_be_positive = 0

    tweet = Variable(X).cuda()
    label = y.view(-1)   
    output = model(tweet)
    a, predicted = torch.max(output.data, 1)

#     total += label.size(0)
#     correct += (predicted == label).sum()
#     positive += predicted.sum()
#     should_be_positive += label.sum()

    labels = []
    outputs = []

    for i in range(len(predicted)):
        
        labels.append(label[i])
        outputs.append(output.data[i])
        
        total += 1
        
        if predicted[i] == label[i]:
            correct += 1
            
        if predicted[i] == 1:
            positive += 1
            
        if label[i] == 1:
            should_be_positive += 1
        
        if predicted[i] == label[i] and label[i] == 1:
            correct_positive += 1


    precision = correct_positive / positive
    recall = correct_positive / should_be_positive
    f_score = 2 / ( (1 / precision) + (1 / recall) )


    print('%s Accuracy: %d %%' % (model_name, 100 * correct / total)) 
    print('%s F-score: %f' % (model_name, f_score)) 
    #print('%s roc_auc_score: %f\n' % (model_name, roc_auc_score(np.array(labels), np.array(outputs))))
                       
    del tweet
    del label
    del output
    


In [22]:
score_model(rnn, X_train_vec_seq, y_train_torch, 'Train')
score_model(rnn, X_test_vec_seq, y_test_torch, 'test')

Train Accuracy: 87 %
Train F-score: 0.871012
test Accuracy: 64 %
test F-score: 0.647520
